# Replica Set

A [ReplicaSet](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/)'s purpose is to maintain a stable set of replica Pods running at any given time.

ReplicaSet的目的是讓Pods副本在任何時間都能維持同一狀態

> 在minikube的環境下，access Cluster IP或是 access Pod IP的時候要用 minikube ssh, nodeIP 着不需要 minikubessh

In [ ]:
from IPython.display import Image
Image(url="./img/replicaset.jpg", width=300, height=300)

## 什麼時候用ReplicaSet

A ReplicaSet ensures that a sepecified nubmer of pod replicas are running at any given time, it is suitable for Stateless application.

ReplicaSet可以保持指定數量的Pods副本在任何時間繼續運行.

## 什麼是無狀態應用 Stateless

Stateless 是指這個應用服務或功能，不因任何狀態而改變回傳的資料。
- eg: 沒有資料庫的static web

## 什麼是有狀態應用  StateFul

Stateful 則與 Stateless 相反，會因狀態而改變回傳的資料，像是Request的次數。
- eg: 有資料庫的 wordpress

## Replica Set

而ReplicaSet 正適合與建立stateless服務，因爲Pod在消滅後再建立的話，是不保存之前的狀態的。

ReplicaSet:
- 管理一群Pods
- 透過 Selector 選擇那些Pod 需要進行管理
- 每個 ReplicaSet 可以根據需求建立或刪除Pod使得副本達到期望值
- 利用template建立Container


In [116]:
# 確保你的minikube有啟動
! [[ -z $(kubectl get nodes) ]] && minikube start 
# 列印目前k8s節點
! kubectl get nodes                    

NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   8h    v1.25.3


In [134]:
# Get the documentation of the resource and its fields
! kubectl explain replicaset

KIND:     ReplicaSet
VERSION:  apps/v1

DESCRIPTION:
     ReplicaSet ensures that a specified number of pod replicas are running at
     any given time.

FIELDS:
   apiVersion	<string>
     APIVersion defines the versioned schema of this representation of an
     object. Servers should convert recognized schemas to the latest internal
     value, and may reject unrecognized values. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#resources

   kind	<string>
     Kind is a string value representing the REST resource this object
     represents. Servers may infer this from the endpoint the client submits
     requests to. Cannot be updated. In CamelCase. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#types-kinds

   metadata	<Object>
     If the Labels of a ReplicaSet are empty, they are defaulted to be the same
     as the Pod(s) that the ReplicaSet manages. Standard object's metadata

In [136]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.replicas

KIND:     ReplicaSet
VERSION:  apps/v1

FIELD:    replicas <integer>

DESCRIPTION:
     Replicas is the number of desired replicas. This is a pointer to
     distinguish between explicit zero and unspecified. Defaults to 1. More
     info:
     https://kubernetes.io/docs/concepts/workloads/controllers/replicationcontroller/#what-is-a-replicationcontroller


In [137]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.selector

KIND:     ReplicaSet
VERSION:  apps/v1

RESOURCE: selector <Object>

DESCRIPTION:
     Selector is a label query over pods that should match the replica count.
     Label keys and values that must match in order to be controlled by this
     replica set. It must match the pod template's labels. More info:
     https://kubernetes.io/docs/concepts/overview/working-with-objects/labels/#label-selectors

     A label selector is a label query over a set of resources. The result of
     matchLabels and matchExpressions are ANDed. An empty label selector matches
     all objects. A null label selector matches no objects.

FIELDS:
   matchExpressions	<[]Object>
     matchExpressions is a list of label selector requirements. The requirements
     are ANDed.

   matchLabels	<map[string]string>
     matchLabels is a map of {key,value} pairs. A single {key,value} in the
     matchLabels map is equivalent to an element of matchExpressions, whose key
     field is "key", the operator is "In", and th

In [135]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.template

KIND:     ReplicaSet
VERSION:  apps/v1

RESOURCE: template <Object>

DESCRIPTION:
     Template is the object that describes the pod that will be created if
     insufficient replicas are detected. More info:
     https://kubernetes.io/docs/concepts/workloads/controllers/replicationcontroller#pod-template

     PodTemplateSpec describes the data a pod should have when created from a
     template

FIELDS:
   metadata	<Object>
     Standard object's metadata. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#metadata

   spec	<Object>
     Specification of the desired behavior of the pod. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#spec-and-status



In [140]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.template.spec.containers

KIND:     ReplicaSet
VERSION:  apps/v1

RESOURCE: containers <[]Object>

DESCRIPTION:
     List of containers belonging to the pod. Containers cannot currently be
     added or removed. There must be at least one container in a Pod. Cannot be
     updated.

     A single application container that you want to run within a pod.

FIELDS:
   args	<[]string>
     Arguments to the entrypoint. The container image's CMD is used if this is
     not provided. Variable references $(VAR_NAME) are expanded using the
     container's environment. If a variable cannot be resolved, the reference in
     the input string will be unchanged. Double $$ are reduced to a single $,
     which allows for escaping the $(VAR_NAME) syntax: i.e. "$$(VAR_NAME)" will
     produce the string literal "$(VAR_NAME)". Escaped references will never be
     expanded, regardless of whether the variable exists or not. Cannot be
     updated. More info:
     https://kubernetes.io/docs/tasks/inject-data-application/define-co

In [115]:
# 所使用的範例replicaset.yaml
! cat replicaset.yaml             

apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: hostname
  labels:
    app: hostname
spec:
  # modify replicas according to your case
  replicas: 3
  selector:
    matchLabels:
      tier: frontend
  template:
    metadata:
      labels:
        tier: frontend
    spec:
      containers:
      - name: hostname
        image: docker.io/brokenpen/hostname:v1

- yaml
  - apiVersion
  - kind
  - metadata
  - spec
     - replicas # 指定要維持多少分replica
     - selector # 選擇器，根據 Pod 的 Labels
     - template # container template

In [117]:
# 透過yaml 建立 replicaSet
! kubectl apply -f replicaset.yaml            # k apply -f replicaset.yaml

replicaset.apps/hostname unchanged


In [118]:
# 列印目前 replicaset 
! kubectl get replicaset

NAME       DESIRED   CURRENT   READY   AGE
hostname   3         3         3       8h


` kubectl get replicaset `
- `NAME` ReplicaSet 的名稱
- `DESIRED` 目標 Replica 數量
- `CURRENT` 目前 Replica 數量
- `READY` 當前 Ready的 Replica 數量
---


In [120]:
# 列印目前 rs(replicaset) 並且顯示 Labels
! kubectl get rs --show-labels

NAME       DESIRED   CURRENT   READY   AGE   LABELS
hostname   3         3         3       8h    app=hostname


In [121]:
# 列印目前 labels 有標註 tier:frontend 的 Pods
! kubectl get pods --selector=tier=frontend --show-labels -o wide

NAME             READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES   LABELS
hostname-5wdst   1/1     Running   0          8h    172.17.0.3   minikube   <none>           <none>            tier=frontend
hostname-74c7b   1/1     Running   0          8h    172.17.0.5   minikube   <none>           <none>            tier=frontend
hostname-dp5xn   1/1     Running   0          8h    172.17.0.4   minikube   <none>           <none>            tier=frontend


In [113]:
! kubectl get pods \
  --selector=tier=frontend \
  --no-headers \
  -o custom-columns=":status.podIP"

172.17.0.3
172.17.0.5
172.17.0.4


In [ ]:
# Ctrl+Shift+L
# minikube ssh
# ! curl 172.17.0.3:8080

curl: (7) Failed to connect to 172.17.0.3 port 8080 after 3062 ms: No route to host


In [ ]:
# 列印目前 Pods 和其Labels
! kubectl get pods --show-labels

NAME             READY   STATUS    RESTARTS   AGE    LABELS
hostname-tfndl   1/1     Running   0          3m4s   tier=frontend
hostname-x6bx8   1/1     Running   0          3m4s   tier=frontend
hostname-z57gd   1/1     Running   0          3m4s   tier=frontend


In [122]:
! kubectl get pods \
  --selector=tier=frontend \
  --no-headers \
  -o custom-columns=":metadata.name"

hostname-5wdst
hostname-74c7b
hostname-dp5xn


In [126]:
! kubectl delete pods --selector=tier=frontend
! echo;
! kubectl get pods --selector=tier=frontend
! echo;

pod "hostname-6lz97" deleted
pod "hostname-7qbgr" deleted
pod "hostname-8st87" deleted
pod "hostname-94nh4" deleted
pod "hostname-c5j2h" deleted
pod "hostname-thwf8" deleted

NAME             READY   STATUS    RESTARTS   AGE
hostname-b72mq   1/1     Running   0          32s
hostname-gch2c   1/1     Running   0          32s
hostname-lfhhr   1/1     Running   0          32s



In [ ]:
! kubectl get pods --selector=tier=frontend 

NAME             READY   STATUS        RESTARTS   AGE
hostname-82cwq   1/1     Running       0          11s
hostname-dffcp   1/1     Running       0          11s
hostname-tfndl   1/1     Terminating   0          3m22s
hostname-thpsr   1/1     Running       0          11s
hostname-x6bx8   1/1     Terminating   0          3m22s
hostname-z57gd   1/1     Terminating   0          3m22s


In [129]:
! kubectl scale --help

Set a new size for a deployment, replica set, replication controller, or
stateful set.

 Scale also allows users to specify one or more preconditions for the scale
action.

 If --current-replicas or --resource-version is specified, it is validated
before the scale is attempted, and it is guaranteed that the precondition holds
true when the scale is sent to the server.

Examples:
  # Scale a replica set named 'foo' to 3
  kubectl scale --replicas=3 rs/foo
  
  # Scale a resource identified by type and name specified in "foo.yaml" to 3
  kubectl scale --replicas=3 -f foo.yaml
  
  # If the deployment named mysql's current size is 2, scale mysql to 3
  kubectl scale --current-replicas=2 --replicas=3 deployment/mysql
  
  # Scale multiple replication controllers
  kubectl scale --replicas=5 rc/foo rc/bar rc/baz
  
  # Scale stateful set named 'web' to 3
  kubectl scale --replicas=3 statefulset/web

Options:
    --all=false:
	Select all resources in the namespace of the specified resource t

In [143]:
! kubectl scale --replicas=5 replicaset hostname

replicaset.apps/hostname scaled


In [144]:
! kubectl get pods --selector=tier=frontend 

NAME             READY   STATUS    RESTARTS   AGE
hostname-b72mq   1/1     Running   0          12m
hostname-c4r4j   1/1     Running   0          11s
hostname-gch2c   1/1     Running   0          12m
hostname-knvqw   1/1     Running   0          11s
hostname-lfhhr   1/1     Running   0          12m


In [145]:
! kubectl scale --replicas=4 replicaset hostname
! kubectl get pods --selector=tier=frontend 

replicaset.apps/hostname scaled
NAME             READY   STATUS        RESTARTS   AGE
hostname-b72mq   1/1     Running       0          12m
hostname-c4r4j   1/1     Running       0          16s
hostname-gch2c   1/1     Running       0          12m
hostname-knvqw   1/1     Terminating   0          16s
hostname-lfhhr   1/1     Running       0          12m


In [146]:
! kubectl scale --replicas=2 replicaset hostname
! kubectl get pods --selector=tier=frontend 

replicaset.apps/hostname scaled
NAME             READY   STATUS        RESTARTS   AGE
hostname-b72mq   1/1     Running       0          12m
hostname-c4r4j   1/1     Terminating   0          25s
hostname-gch2c   1/1     Terminating   0          12m
hostname-knvqw   1/1     Terminating   0          25s
hostname-lfhhr   1/1     Running       0          12m


In [ ]:
#

In [149]:
# Autoscale
! kubectl autoscale rs hostname --max=10 --min=2 --cpu-percent=100

Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "hostname" already exists


In [192]:
# 可是 minikube 好像是捉不到targets 的 實際 cpu-percent 
# 所以在我們的workshop裡面，沒有用
! kubectl get HorizontalPodAutoscaler

NAME       REFERENCE             TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
hostname   ReplicaSet/hostname   <unknown>/50%   4         10        2          34s


In [193]:
! kubectl get hpa

NAME       REFERENCE             TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
hostname   ReplicaSet/hostname   <unknown>/50%   4         10        2          35s


In [182]:
! kubectl delete hpa hostname
! kubectl autoscale --max=10 --min=4 --cpu-percent=50 -f replicaset.yaml

horizontalpodautoscaler.autoscaling "hostname" deleted
horizontalpodautoscaler.autoscaling/hostname autoscaled


In [197]:
! kubectl get pods --selector=tier=frontend 

NAME             READY   STATUS        RESTARTS   AGE
hostname-b72mq   1/1     Terminating   0          18m
hostname-kcpnm   1/1     Terminating   0          29s
hostname-lf287   1/1     Terminating   0          29s
hostname-lfhhr   1/1     Terminating   0          18m


In [198]:
# 刪除所有 replicaset 
! kubectl delete replicaset --all

No resources found


In [ ]:
# 刪除所有 replicaset 
! kubectl delete hpa --all